In [1]:
from nltk import pos_tag
import pandas as pd
import numpy as np 
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import re
import nltk 
from itertools import chain
from wordcloud import WordCloud, STOPWORDS
import matplotlib.pyplot as plt
import seaborn as sns
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import LdaModel, CoherenceModel, HdpModel
import spacy
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\giorg\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\giorg\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
pd.set_option('display.max_colwidth', None)
df = pd.read_csv(r'C:\Users\giorg\clean_text.csv', encoding = 'utf-8-sig')

# Pre-Processing

In [4]:
text = df['text']
text = list(text)


In [5]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

text_words = list(sent_to_words(text))

print(text_words[:1])

[['folks', 'paste', 'cytokine', 'storm', 'pfizerbiontech']]


In [6]:
bigram = gensim.models.Phrases(text_words, min_count=1, threshold=10) 
trigram = gensim.models.Phrases(bigram[text_words], threshold=10)  

bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

print(trigram_mod[bigram_mod[text_words[0]]])

['folks', 'paste', 'cytokine', 'storm', 'pfizerbiontech']


In [7]:
def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [8]:
# Form Bigrams
text_words_bigrams = make_bigrams(text_words)

nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
text_lemmatized = lemmatization(text_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(text_lemmatized[:1])

[['folk', 'paste', 'cytokine', 'storm', 'pfizerbiontech']]


In [9]:
id2word = corpora.Dictionary(text_lemmatized)

# Create Corpus
texts = text_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1)]]


In [10]:
id2word[0]

'cytokine'

In [11]:
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

[[('cytokine', 1),
  ('folk', 1),
  ('paste', 1),
  ('pfizerbiontech', 1),
  ('storm', 1)]]

# Basic HDP model

In [12]:
hdp_model = HdpModel(corpus=corpus, id2word=id2word)
hdp_model.show_topics()

[(0,
  '0.059*vaccine + 0.035*moderna + 0.034*covid + 0.014*dose + 0.011*covaxin + 0.009*covidvaccine + 0.009*sputnikv + 0.009*today + 0.008*pfizer + 0.007*shoot + 0.007*second + 0.006*day + 0.006*india + 0.005*shot + 0.005*pfizerbiontech + 0.004*vaccination + 0.004*amp + 0.004*jab + 0.004*coronavirus + 0.004*sinovac'),
 (1,
  '0.032*vaccine + 0.017*covid + 0.015*moderna + 0.011*covaxin + 0.010*dose + 0.006*today + 0.005*sputnikv + 0.005*covidvaccine + 0.004*pfizer + 0.004*pfizerbiontech + 0.004*india + 0.003*vaccination + 0.003*thank + 0.003*amp + 0.003*shoot + 0.003*day + 0.003*second + 0.002*shot + 0.002*sinopharm + 0.002*first'),
 (2,
  '0.015*vaccine + 0.008*covid + 0.007*moderna + 0.004*covaxin + 0.003*dose + 0.003*pfizerbiontech + 0.003*sputnikv + 0.002*covidvaccine + 0.002*today + 0.002*pfizer + 0.001*day + 0.001*vaccination + 0.001*shoot + 0.001*india + 0.001*sinopharm + 0.001*amp + 0.001*second + 0.001*shot + 0.001*sinovac + 0.001*coronavirus'),
 (3,
  '0.014*vaccine + 0.007*

In [15]:
hdp_cm = CoherenceModel(model = hdp_model, corpus = corpus, dictionary= id2word, texts = text_lemmatized, coherence = "c_v")
hdp_CM = hdp_cm.get_coherence()
print('\nCoherence Score: ', hdp_CM)


Coherence Score:  0.20384487982077054
